### Raw Data Download Notebook

*Important Note:* This notebook runs using a different environment than this project's computational notebook. ObsPy is required, along with numpy and matplotlib

Zali et al. 2024 pulls horizontal seismic data from a single horizontal component from a single station, demeaned and detrended but not converted from instrument response. The data is pulled from March 12 to June 24, ~104 days, beginning 7 days before the Geldingadalir eruption began. The data is also downsampled to the minimum sampling rate necessary to observe the local volcanic tremor.

------------------

This notebook will download 100 days of data, beginning May 16 and ending August 24, from sensor AV.GSTD of the Alaska Volcano Observatory's network, located on the Great Sitkin Volcano. The Great Sitkin had an explosive eruption at 05:04 (UTC) May 26, 2021, followed by lava flow in mid-July https://avo.alaska.edu/eruption/great-sitkin-2021-05.

In [3]:
import obspy
from obspy import UTCDateTime as utc
from obspy.clients.fdsn import Client
client = Client('IRIS')
import numpy as np
import os

In [4]:
def process(st, buffer, freq, max_target_frequency): #freq is the original sampling frequency
    st.merge(fill_value='interpolate')
    tr = st[0].copy()

### Data Description

The seismic data is downloaded from the IRIS/Earthscope Database using ObsPy. The data downloads as a trace, an array of data with attached metadata, which is then packaged into a stream, which can contain multiple traces. The daily seismic data records are then saved as mseed files, which preserve this data+metadata structure, but requires ObsPy or other specialized software to open.

In [6]:
#creating variables to download data
starttime = utc('2021-05-16T00:00:00')
endtime = starttime + 100 * (60*60*24)

#also add a buffer to both ends to chop off once the data has been filtered
#and downsampled, kind of arbitrary length, 5% of a day (default ObsPy taper length)
buffer = 60*60*24*0.05 #seconds

net = 'AV'
sta = 'GSTD'
loc = '*' #wildcard, generally don't care about location code
cha = 'BHN' #horizontal component, as used in Zali et al

#create folder for numpy streams to go into and initialize filepath
! mkdir data
! mkdir data/raw
filepath = os.getcwd() + '/data/raw/'

#create arrays to save dates
dates = np.array([])

#download the data piecemeal, here by day
for day in range(100):
    tr_length = 24*60*60

    #actually downloading
    st = client.get_waveforms(network=net,
                     station=sta,
                     location=loc,
                     channel=cha,
                     starttime=starttime-buffer,
                     endtime=starttime+buffer+tr_length)

    #instrument sampling rate (hz)
    freq = st[0].stats.sampling_rate

    #merge traces within stream, linearly interpolating any gaps
    st.merge(fill_value='interpolate')
    
    #generate filename, day number in front for convenience of reading in
    name = str(day+1)+'_sitkin.mseed'
    
    #save data as mseed, standard for storing seismic data. Preserves metadata and time series info
    st.write(filepath+name, format='MSEED')

    #adding date
    dates = np.append(dates, starttime.date)

    starttime += tr_length

#save dates list for future use
np.save(filepath+'date_list.csv', dates)

mkdir: data: File exists
mkdir: data/raw: File exists
